In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
pivot_table, index_col = self.get_clean_pivot_table('f__answer_position',remove_low_freq_col=True)

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    entropy_ = data.groupby('responsible')[col].apply(calculate_entropy)
    #entropy_ = entropy_.reset_index()
    #data[col+'_entropy'] = data['responsible'].map(entropy_[col])
    entropy_.plot(kind='bar')
    plt.title(col)
    plt.show()

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    entropy_ = data.groupby('responsible')[col].apply(calculate_entropy)
    entropy_ = entropy_.reset_index()
    pivot_table[col+'_entropy'] = pivot_table['responsible'].map(entropy_.set_index('responsible')[col])

In [ ]:
entropy_columns = [col for col in pivot_table.columns if 'entropy' in col]
pivot_table['entropy__answer_position'] = pivot_table[entropy_columns].fillna(0).mean(1)
entropy_ = pivot_table.groupby('responsible')['entropy__answer_position'].mean()
entropy_.plot(kind='bar')
plt.title('entropy__answer_position')
plt.show()

In [ ]:
# import pandas as pd
# import numpy as np
# from scipy.stats import entropy
# from scipy.stats import entropy
# 
# def jensen_shannon_divergence(p, q):
#     """Compute the Jensen-Shannon divergence between two distributions."""
#     m = 0.5 * (p + q)
#     return 0.5 * entropy(p, m) + 0.5 * entropy(q, m)
# # Sample data
# 
# 
# def compute_kl_divergence(df, col='f__answer_position'):
#     responsibles = df['responsible'].unique()
#     kl_divergence = {}
# 
#     # Compute the overall distribution for each unique value
#     overall_distribution = df[col].value_counts(normalize=True).sort_index()
# 
#     for resp in responsibles:
#         # Distribution for the current responsible
#         resp_distribution = df[df['responsible'] == resp][col].value_counts(normalize=True).sort_index()
# 
#         # Combine distributions to ensure they cover the same range of values
#         combined = pd.concat([resp_distribution, overall_distribution], axis=1, sort=True).fillna(0)
#         resp_distribution = combined.iloc[:, 0]
#         overall_distribution = combined.iloc[:, 1]
# 
#         # Compute KL divergence
#         kl = jensen_shannon_divergence(resp_distribution, overall_distribution)
#         kl_divergence[resp] = kl
# 
#     return kl_divergence
# data = pivot_table[~pd.isnull(pivot_table[col])].copy()
# kl_results = compute_kl_divergence(data, col)
# print(kl_results)
# data['kl'] = data['responsible'].replace(kl_results)

# Function to calculate entropy
def calculate_entropy(column):
    # Value counts normalizes the counts to get probabilities
    prob_distribution = column.value_counts(normalize=True).values
    return entropy(prob_distribution)* len(column)

# Group by responsible and calculate entropy
entropy = data.groupby('responsible')[col].apply(calculate_entropy)
#entropy = entropy.reset_index()
#data['entropy'] = data['responsible'].map(entropy.set_index('responsible')[col])


In [ ]:
from scipy.stats import entropy
data = pivot_table[~pd.isnull(pivot_table[col])].copy()
prob_distribution = data[col].value_counts(normalize=True)
entropy(prob_distribution)

In [ ]:
x = data.groupby('responsible')['entropy'].max()
#x = x.reset_index()
x.plot(kind='bar')